In [6]:
import numpy as np
import igraph as ig
import pandas as pd
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt

from pathlib import Path

import os
import sys  

# set path for pixelator directory 
sys.path.insert(0, "/Users/janrkauert/pixelator/")

# load functions from pixelator
from pixelator.pixeldataset import build_graph, PixelDataset
from pixelator.analysis import polarization_scores

# set path for pixelator directory 
sys.path.insert(1, "/Users/janrkauert/Documents/GitHub/sandbox/PXL00018/")

# load standard workflow single cell functions
from single_cell_functions.line_graph import create_line_graph
from single_cell_functions.local_assortativity import calculate_local_assorativity
from single_cell_functions.plot import plot_mutliple_markers, plot_mutliple_components
from single_cell_functions.global_assortativity import calculate_assortativity_component_wise
from single_cell_functions.similarity_measures import compare_multiple_markers


# get directory with graph eval files
path_name = "/Users/janrkauert/Downloads/Updated_uropods"

In [7]:
# new graph for all data
sample_list = []
pxl_data_list = []

for index_place, file in enumerate(sorted(os.listdir(path_name))):
    
    # get name of current file in directory
    filename = os.fsdecode(file)

    # load marker edge list
    pxl_data = PixelDataset.from_file(path_name + "/" + filename)
    edge_list_df = pxl_data.edgelist   

    edge_list_df.columns = [col.lower() for col in edge_list_df.columns]

    # build the graph
    mega_graph = build_graph(
                edgelist=edge_list_df,
                add_marker_counts=True,
                simplify=False,
                use_full_bipartite=False,
            )
    
    # add filename as a vertex attribute
    mega_graph.es["sample"] = [filename] * mega_graph.ecount()

    sample_list.append(mega_graph)
    pxl_data_list.append(pxl_data)

    print(filename)

Uropod_CD54_Rantes_coated_S3_001.dataset.pxl
Uropod_control_S1_001.dataset.pxl


In [10]:
# select sample to work on
sample_index = 1
marker = "CD50"

# create component dict
comp_dict = dict(zip(pxl_data_list[sample_index].edgelist["upia"].values.tolist(), pxl_data_list[sample_index].edgelist["component"].values.tolist()))

# subgraphs of the selected sample
graph_component_subgraphs = sample_list[sample_index].components().subgraphs()

# add component name
for comp in graph_component_subgraphs:
    comp.vs["component"] = comp_dict[comp.vs[0]["name"]]

# get polarization scores for the selected marker
pol_df = pxl_data_list[sample_index].polarization
pol_df = pol_df.loc[pol_df['marker'] == marker]

# get ten most polarized components names
pol_components = pol_df.sort_values(by="morans_i", ascending=False)["component"][:200].values.tolist()
pol_components = set(pol_components)

# get the subgraphs belonging to the selected component names
top_polarised_components = [comp for comp in graph_component_subgraphs if comp_dict[comp.vs[0]["name"]] in pol_components and comp.vcount() > 1000 and comp.vcount() < 3000]

In [11]:
marker_names = ["CD37", "CD43", "CD44", "CD50", "CD54", "CD82", "CD102", "CD162", "CD11a", "CD18","mIgG1"]

# create new dataframe to keep track of similarity measures
similarity_measure_df = pd.DataFrame()
list_dfs = []

# iterate through the first 100 components in the current sample
for comp_index, component in enumerate(top_polarised_components[:100]):
    
    local_assort_score_df = pd.DataFrame()
    
    for marker in marker_names:

        loc_rounded = calculate_local_assorativity(component, marker)
        local_assort_score_df[marker] = loc_rounded       
            
    local_assort_score_df.to_csv("/Users/janrkauert/Documents/GitHub/sandbox/PXL00018/single_cells/code_paper/paper_data/S1/" + str(comp_index))